case1: converting chain/residue id
This case shows a sample to rebnumber antibody in IMGT scheme

In [46]:
from DPMacro import hmt_FvProcessor,impute_beta
p=hmt_FvProcessor('i')
p.transform('sample/1bey_trunc_HL_Fv_ele.pdb')
p.renumber_structure()
impute_beta(p.object,lambda x: 1 if 'CDR' in x.xtra['CDR'] else 0)
p.write_built_structure('sample/renumber.pdb')

See renumber.png for detailed changes in residue id.

The CDR is annotated at the sametime

<img src='sample/renumber.png' width=480>

case1.1: extract H-L-A from a large system

In [1]:
from DPMacro import FvStructureProcesser as fv
from DPMacro import util as u
s=u.read_in('sample/7wn8_imgt.pdb')
f=fv(s)
f.parse_structure('i')
f.__dict__

/home/dzf/miniconda3/envs/SE3nv/lib/python3.9/site-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(


{'object': <Structure id=sample/7wn8_imgt.pdb>,
 'scheme': 'i',
 'chain_list': {'A': 'QLLFNKTKSVEFTFCNDTVVIPCFVTNMEAQNTTEVYVKWKFKGRDIYTFDGALNKSTVPTDFSSAKIEVSQLLKGDASLKMDKSDAVSHTGNYTCEVTELTREGETIIELKYRVV',
  'C': 'QLLFNKTKSVTVVIPCFVTNMEAQNTTEVYVKWKFKGRDIYTFDGALNKSTVPTDFSSAKIEVSQLLKGDASLKMDKSDAVSHTGNYTCEVTELTREGETIIEL',
  'H': 'QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYGISWVRQAPGQGLEWMGWISAYNGNTNYAQKLQGRVTMTTDTSTSTAYMELRSLRSDDTAVYYCARGTVYFDYWGQGTLVTVSSASTKGPSVFPLAPGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKA',
  'L': 'EIVLTQSPATLSLSPGERATLSCRASQSLSRNHLAWYQQKPGQPPRLLIHWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQHTQNTWTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGE',
  'P': 'QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYGISWVRQAPGQGLEWMGWISAYNGNTNYAQKLQGRVTMTTDTSTSTAYMELRSLRSDDTAVYYCARGTVYFDYWGQGTLVTVSSASTKGPSVFPLAPAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKAE',
  'Q': 'EIVLTQSPATLS

In [ ]:
m=u.Model(0)
m.add(f.splited_chains['A']['Ag0'].copy())
u.extract_hetatm(m['A'],inplace=True)
m.add(f.splited_chains['H']['Fv0'].copy())
m['Fv0'].id='H'
m.add(f.splited_chains['L']['Fv0'].copy())
m['Fv0'].id='L'
u.write_out(m,'sample/7wn8_clean.pdb')

case2:calculate features and impute B-factors
This case shows a sample to calculate all supported feature in this suite.

In [47]:
from DPMacro import PPSExtractor,impute_beta,write_out
p=PPSExtractor()
features=p.transform('sample/1bey_trunc_HL_Fv_ele.pdb')
print(p.frame.columns)

Index(['model', 'chain', 'residue', 'resname', 'object', 'SS_DSSP',
       'EXP_DSSP_ASA', 'PHI_DSSP', 'PSI_DSSP', 'DSSP_INDEX',
       'NH_O_1_RELIDX_DSSP', 'NH_O_1_ENERGY_DSSP', 'O_NH_1_RELIDX_DSSP',
       'O_NH_1_ENERGY_DSSP', 'NH_O_2_RELIDX_DSSP', 'NH_O_2_ENERGY_DSSP',
       'O_NH_2_RELIDX_DSSP', 'O_NH_2_ENERGY_DSSP', 'EXP_DSSP_RASA',
       'PHI_INTERNAL', 'PSI_INTERNAL', 'OMG_INTERNAL', 'TAU_INTERNAL',
       'CHI1_INTERNAL', 'EXP_HSE_B_U', 'EXP_HSE_B_D', 'EXP_HSE_RATIO',
       'SASA_INTERNAL', 'hydrophobicity', 'static_charge', 'i_sasa',
       'georged_i_sasa', 'salt_bridge', 'salted_charge',
       'salted_hydrophobicity', 'f_sasa', 'surface', 'PFWY', 'PRKDE', 'PL',
       'PLIV', 'NHydro', 'NCharg', 'psh', 'ppc', 'pnc'],
      dtype='object')


In [8]:
impute_beta(p.object,lambda x:x.xtra['f_sasa'])
write_out(p.object,'sample/sasa.pdb')
impute_beta(p.object,lambda x:x.xtra['psh'])
write_out(p.object,'sample/psh.pdb')

numeric features could  be imputed into beta colums and is expeditious for versualization

upper:fsasa(surface) lower:psh(hydrophobicity)


<img src='sample/fsasa.png' width=480>


<img src='sample/psh.png' width=480>


Note:

the base class ResidueFeatureExtractor is used for read/operate/save structure & features,
it contains a frame (pd.DataFrame) to record features and a object to record structure,
the feature is impute into the .xtra of each Residue in the object, as well.

In [42]:
p.frame.columns
#p.frame is a DataFrame with all calculated feature

Index(['model', 'chain', 'residue', 'resname', 'object', 'anarci_order',
       'Fv_annotation', 'chain_type', 'CDR'],
      dtype='object')

In [28]:
p.object
##p.object is a Bio.PDB.Strucrture, I'll expatiate it later

<Structure id=sample/1bey_trunc_HL_Fv_ele.pdb>

case3: manipulate structures
This case shows a sample to extract ligand and pockets(residue with 6A of ligands) 

In [33]:
from DPMacro import PocketExtractor,reslist_to_guicode
p=PocketExtractor()
p.load_structure('sample/1uui.pdb')
p.set_ligand(p.ligands[0])
p.save('sample/1uui')

In [39]:
#convert pocket to selection code in pymol
print(reslist_to_guicode(p.pocket))

'( (chain B and resi 39 ) or (chain B and resi 27 ) or (chain B and resi 38 ) or (chain B and resi 26 ) or (chain B and resi 40 ) or (chain B and resi 22 ) or (chain B and resi 36 ) or (chain B and resi 29 ) or (chain B and resi 41 ) or (chain B and resi 28 ) or (chain B and resi 35 ) or (chain B and resi 24 ) or (chain B and resi 37 ) or (chain B and resi 23 ) )'

the pocket and ligand are saved into separated pdb file.

<img src='sample/pockets.png' width=480>

Note:

the structure object is PDB.Structure
it is a hierarchical dict
when refer to chain H's residue 1's CA atom in frame 0:

In [23]:
a1=p.object[0]['H'][(' ',1,' ')]['CA']
type(a1)

Bio.PDB.Atom.Atom

Atom stores all information we want:

In [24]:
a1.__dict__

{'level': 'A',
 'parent': <Residue GLN het=  resseq=1 icode= >,
 'name': 'CA',
 'fullname': ' CA ',
 'coord': array([ 6.897, 79.705, 43.426], dtype=float32),
 'bfactor': 0.6888023922624382,
 'occupancy': 1.0,
 'altloc': ' ',
 'full_id': ('sample/1bey_trunc_HL_Fv_ele.pdb',
  0,
  'H',
  (' ', 1, ' '),
  ('CA', ' ')),
 'id': 'CA',
 'disordered_flag': 0,
 'anisou_array': None,
 'siguij_array': None,
 'sigatm_array': None,
 'serial_number': 2,
 'xtra': {},
 'element': 'C',
 'mass': 12.0107,
 'pqr_charge': None,
 'radius': None,
 '_sorting_keys': {'N': 0, 'CA': 1, 'C': 2, 'O': 3},
 'sasa': 0.0}